Notebook designed for exploratory analysis and data treatment of datasets

In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [32]:
df_ratings = pd.read_csv('ratings_small.csv')
df_movies = pd.read_csv('movies_metadata.csv', low_memory=False)
df_ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [33]:
df_movies.head(10)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [34]:
df_movies['id_num'] = pd.to_numeric(df_movies['id'], errors='coerce')

# ver se tem muita coisa que não converteu
print("IDs não numéricos:", df_movies['id_num'].isna().sum())


IDs não numéricos: 3


In [35]:
df_movies = df_movies[~df_movies['id_num'].isna()].copy()

# agora sim, id como int
df_movies['id'] = df_movies['id_num'].astype(int)

# não precisamos mais de id_num
df_movies = df_movies.drop(columns=['id_num'])

print("dtype id depois:", df_movies['id'].dtype)

dtype id depois: int64


In [36]:

print("movieId nunique em df_ratings_filtered:", df_ratings['movieId'].nunique())
print("id nunique em df_movies:", df_movies['id'].nunique())

print("dtype movieId:", df_ratings['movieId'].dtype)
print("dtype id     :", df_movies['id'].dtype)

# garantir que são inteiros (se der erro, já sabemos que tem sujeira)
df_ratings['movieId'] = df_ratings['movieId'].astype(int)
df_movies['id'] = df_movies['id'].astype(int)

# interseção
common_ids = set(df_ratings['movieId']).intersection(df_movies['id'])
print("Quantidade de IDs em comum:", len(common_ids))


movieId nunique em df_ratings_filtered: 9066
id nunique em df_movies: 45433
dtype movieId: int64
dtype id     : int64
Quantidade de IDs em comum: 2830


In [37]:
# 1) Drop de colunas dos dois DFs originais
df_movies = df_movies.drop(
    ['adult', 'video', 'imdb_id', 'original_language', 'revenue', 'status',
     'spoken_languages', 'belongs_to_collection', 'budget', 'homepage',
     'runtime', 'production_countries'],
    axis=1,
    errors='ignore'
)

df_ratings = df_ratings.drop(['timestamp'], axis=1, errors='ignore')

# 2) Contar avaliações por filme e por usuário
ratings_counts_movies = df_ratings['movieId'].value_counts()
ratings_counts_users  = df_ratings['userId'].value_counts()

# filmes com >= 50 avaliações
valid_movies = ratings_counts_movies[ratings_counts_movies >= 50].index

# usuários com >= 50 avaliações
valid_users = ratings_counts_users[ratings_counts_users >= 50].index

# 3) APLICAR OS DOIS FILTROS NO MESMO DF (interseção)
df_ratings_filtered = df_ratings[
    df_ratings['movieId'].isin(valid_movies) &
    df_ratings['userId'].isin(valid_users)
].reset_index(drop=True)

# 4) Atualizar df_movies para conter só filmes que aparecem em df_ratings_filtered
df_movies_filtered = df_movies[
    df_movies['id'].isin(df_ratings_filtered['movieId'].unique())
].reset_index(drop=True)

print("Before treatment:")
print("  Users :", df_ratings['userId'].nunique())
print("  Movies:", df_movies['id'].nunique())

print("\nAfter treatment:")
print("  Users :", df_ratings_filtered['userId'].nunique())
print("  Movies:", df_movies_filtered['id'].nunique())


Before treatment:
  Users : 671
  Movies: 45433

After treatment:
  Users : 427
  Movies: 247


In [38]:
df_movies.columns
df_movies.head(10)

,genres,id,original_title,overview,popularity,poster_path,production_companies,release_date,tagline,title,vote_average,vote_count
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,NaN,Toy Story,7.7,5415.0
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
5,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",949,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...",1995-12-15,A Los Angeles Crime Saga,Heat,7.7,1886.0
6,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",11860,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",1995-12-15,You are cordially invited to the most surprisi...,Sabrina,6.2,141.0
7,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",45325,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",1995-12-22,The Original Bad Boys.,Tom and Huck,5.4,45.0
8,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",9091,Sudden Death,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1995-12-22,Terror goes into overtime.,Sudden Death,5.5,174.0
9,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",710,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...",1995-11-16,No limits. No fears. No substitutes.,GoldenEye,6.6,1194.0


In [39]:
import ast

df_movies['genres'] = df_movies['genres'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else []
)



In [40]:
df_movies['genres'] = df_movies['genres'].apply(
    lambda lista: " ".join(g['name'] for g in lista) if isinstance(lista, list) else ""
)
df_movies[['id', 'title', 'genres']].head(10)

,id,title,genres
0,862,Toy Story,Animation Comedy Family
1,8844,Jumanji,Adventure Fantasy Family
2,15602,Grumpier Old Men,Romance Comedy
3,31357,Waiting to Exhale,Comedy Drama Romance
4,11862,Father of the Bride Part II,Comedy
5,949,Heat,Action Crime Drama Thriller
6,11860,Sabrina,Comedy Romance
7,45325,Tom and Huck,Action Adventure Drama Family
8,9091,Sudden Death,Action Adventure Thriller
9,710,GoldenEye,Adventure Action Thriller


In [41]:
df_movies['genres'].sample(60).tolist()


['Fantasy Horror Mystery',
 'Western',
 'Drama',
 'Science Fiction Action Adventure Comedy',
 'Comedy Romance Drama',
 'Comedy',
 '',
 'Comedy Family',
 'Animation Fantasy',
 'Music',
 'Drama Fantasy Foreign Romance Science Fiction',
 'Thriller Mystery',
 'Drama',
 'Thriller Mystery Foreign',
 'Comedy Documentary',
 'Comedy Thriller Mystery',
 'Documentary',
 '',
 'Action Drama',
 'Crime Drama',
 'Crime Drama Foreign',
 '',
 'Drama',
 'Documentary Comedy Drama',
 'Documentary',
 'Comedy Crime Drama',
 'Drama',
 'Drama',
 'History Romance Western',
 'Comedy Drama',
 'Action Animation',
 'Western',
 'Drama',
 'Thriller',
 'Comedy',
 'Horror Science Fiction Thriller',
 'Adventure Fantasy',
 'Drama Action Adventure',
 'TV Movie Animation Family Mystery Comedy',
 'Action Drama',
 'Thriller Drama',
 'Comedy',
 'Comedy',
 'Thriller Western Action Drama',
 'Drama Comedy Romance',
 'Mystery Thriller Drama',
 'Crime Drama Mystery',
 'Comedy Drama',
 'Comedy Foreign',
 'Drama Family Romance',
 'D

In [42]:
for x in df_movies['genres'].head(5):
    print(repr(x))

'Animation Comedy Family'
'Adventure Fantasy Family'
'Romance Comedy'
'Comedy Drama Romance'
'Comedy'


In [43]:
GENRES_VALIDOS = {
    'Action','Adventure','Animation','Comedy','Crime','Documentary','Drama',
    'Family','Fantasy','History','Horror','Music','Mystery','Romance',
    'Science Fiction','TV Movie','Thriller','War','Western'
}
def limpar_generos(s: str):
    generos_encontrados = []
    for genero in GENRES_VALIDOS:
        if genero in s:  
            generos_encontrados.append(genero)
    return generos_encontrados

df_movies['genres_clean'] = df_movies['genres'].apply(limpar_generos)


In [44]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded = mlb.fit_transform(df_movies['genres_clean'])

df_genres = pd.DataFrame(encoded, columns=mlb.classes_)
df_movies = pd.concat([df_movies, df_genres], axis=1)
df_movies.head(10)

,genres,id,original_title,overview,popularity,poster_path,production_companies,release_date,tagline,title,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,Animation Comedy Family,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,NaN,Toy Story,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adventure Fantasy Family,8844.0,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,Roll the dice and unleash the excitement!,Jumanji,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Romance Comedy,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Comedy Drama Romance,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,Friends are the people who let you be yourself...,Waiting to Exhale,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Comedy,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Action Crime Drama Thriller,949.0,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...",1995-12-15,A Los Angeles Crime Saga,Heat,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,Comedy Romance,11860.0,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",1995-12-15,You are cordially invited to the most surprisi...,Sabrina,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,Action Adventure Drama Family,45325.0,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",1995-12-22,The Original Bad Boys.,Tom and Huck,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Action Adventure Thriller,9091.0,Sudden Death,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1995-12-22,Terror goes into overtime.,Sudden Death,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,Adventure Action Thriller,710.0,GoldenEye,James Bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...",1995-11-16,No limits. No fears. No substitutes.,GoldenEye,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [45]:
df_movies['id'].is_unique

False

In [46]:
dup_ids = df_movies.loc[df_movies['id'].duplicated(), 'id'].unique()
print(len(dup_ids), "ids duplicados")

df_movies[df_movies['id'].isin(dup_ids)].sort_values('id').head(20)

30 ids duplicados


,genres,id,original_title,overview,popularity,poster_path,production_companies,release_date,tagline,title,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
5865,Comedy Crime Drama Romance Thriller,4912.0,Confessions of a Dangerous Mind,"Television made him famous, but his biggest hi...",11.331072,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",2002-12-30,Some things are better left top secret.,Confessions of a Dangerous Mind,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
33826,Comedy Crime Drama Romance Thriller,4912.0,Confessions of a Dangerous Mind,"Television made him famous, but his biggest hi...",7.645827,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",2002-12-30,Some things are better left top secret.,Confessions of a Dangerous Mind,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9165,Crime Drama Thriller,5511.0,Le Samouraï,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"[{'name': 'Fida cinematografica', 'id': 73}, {...",1967-10-25,There is no solitude greater than that of the ...,Le Samouraï,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7345,Crime Drama Thriller,5511.0,Le Samouraï,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"[{'name': 'Fida cinematografica', 'id': 73}, {...",1967-10-25,There is no solitude greater than that of the ...,Le Samouraï,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
44821,Adventure Fantasy Animation Action Family,10991.0,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,6.480376,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...",2000-07-08,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4114,Adventure Fantasy Animation Action Family,10991.0,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,10.264597,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...",2000-07-08,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14012,Comedy Drama,11115.0,Deal,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"[{'name': 'Andertainment Group', 'id': 2634}, ...",2008-01-29,NaN,Deal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24844,Comedy Drama,11115.0,Deal,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"[{'name': 'Andertainment Group', 'id': 2634}, ...",2008-01-29,NaN,Deal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44826,Adventure Fantasy Animation Science Fiction Fa...,12600.0,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",6.080108,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,[],2001-07-06,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5535,Adventure Fantasy Animation Science Fiction Fa...,12600.0,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",7.072301,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,[],2001-07-06,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [47]:
df_movies = (
    df_movies
    .drop_duplicates(subset='id', keep='first')
    .reset_index(drop=True)
)

df_movies['id'].is_unique

True

In [48]:
import ast

df_movies['production_companies'] = df_movies['production_companies'].fillna('[]')

def extract_company_names(x):
    try:
        if isinstance(x, str):
            x = ast.literal_eval(x)
        if not x:
            return ''
        names = [d.get('name', '') for d in x if isinstance(d, dict)]
        return ' '.join(n.strip() for n in names if n.strip())
    except Exception:
        return ''

df_movies['companies_text'] = df_movies['production_companies'].apply(extract_company_names)

text_cols = ['title', 'original_title', 'genres', 'overview', 'tagline', 'companies_text']

for c in text_cols:
    df_movies[c] = (
        df_movies[c]
        .fillna('')
        .astype(str)
        .str.lower()
        .str.strip()
    )

df_movies.head(10)

,genres,id,original_title,overview,popularity,poster_path,production_companies,release_date,tagline,title,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,companies_text
0,animation comedy family,862.0,toy story,"led by woody, andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,,toy story,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pixar animation studios
1,adventure fantasy family,8844.0,jumanji,when siblings judy and peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,roll the dice and unleash the excitement!,jumanji,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tristar pictures teitler film interscope commu...
2,romance comedy,15602.0,grumpier old men,a family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,still yelling. still fighting. still ready for...,grumpier old men,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,warner bros. lancaster gate
3,comedy drama romance,31357.0,waiting to exhale,"cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,friends are the people who let you be yourself...,waiting to exhale,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,twentieth century fox film corporation
4,comedy,11862.0,father of the bride part ii,just when george banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,just when his world is back to normal... he's ...,father of the bride part ii,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sandollar productions touchstone pictures
5,action crime drama thriller,949.0,heat,"obsessive master thief, neil mccauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...",1995-12-15,a los angeles crime saga,heat,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,regency enterprises forward pass warner bros.
6,comedy romance,11860.0,sabrina,an ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...",1995-12-15,you are cordially invited to the most surprisi...,sabrina,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,paramount pictures scott rudin productions mir...
7,action adventure drama family,45325.0,tom and huck,"a mischievous young boy, tom sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]",1995-12-22,the original bad boys.,tom and huck,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,walt disney pictures
8,action adventure thriller,9091.0,sudden death,international action superstar jean claude van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",1995-12-22,terror goes into overtime.,sudden death,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,universal pictures imperial entertainment sign...
9,adventure action thriller,710.0,goldeneye,james bond must unmask the mysterious head of ...,14.686036,/5c0ovjT41KnYIHYuF4AWsTe3sKh.jpg,"[{'name': 'United Artists', 'id': 60}, {'name'...",1995-11-16,no limits. no fears. no substitutes.,goldeneye,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,united artists eon productions


In [49]:
movie_thresh = 50
user_thresh = 50

def filter_ratings_iterative(ratings, movie_thresh=50, user_thresh=50):
    df = ratings.copy()
    while True:
        prev_shape = df.shape[0]
        # filtrar filmes
        movie_counts = df['movieId'].value_counts()
        valid_movies = movie_counts[movie_counts >= movie_thresh].index
        df = df[df['movieId'].isin(valid_movies)]
        # filtrar usuarios
        user_counts = df['userId'].value_counts()
        valid_users = user_counts[user_counts >= user_thresh].index
        df = df[df['userId'].isin(valid_users)]
        # parar se convergiu
        if df.shape[0] == prev_shape:
            break
    return df

df_filtered = filter_ratings_iterative(df_ratings, movie_thresh, user_thresh)

print('Final: users =', df_filtered['userId'].nunique(),
      ', movies =', df_filtered['movieId'].nunique(),
      ', ratings =', df_filtered.shape[0])

Final: users = 236 , movies = 294 , ratings = 24328


In [50]:
cols_for_tfidf = ['title', 'genres', 'overview', 'tagline', 'companies_text']

df_movies['content'] = df_movies[cols_for_tfidf].agg(' '.join, axis=1).str.strip()
df_movies = df_movies[df_movies['content'] != ''].reset_index(drop=True)



In [51]:
print(df_movies['id'].is_unique)                 # True
print(df_filtered['movieId'].isin(df_movies['id']).mean())

True
0.5420503123972378


In [52]:
df_movies.head()

,genres,id,original_title,overview,popularity,poster_path,production_companies,release_date,tagline,title,...,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,companies_text,content
0,animation comedy family,862.0,toy story,"led by woody, andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,,toy story,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,pixar animation studios,toy story animation comedy family led by woody...
1,adventure fantasy family,8844.0,jumanji,when siblings judy and peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,roll the dice and unleash the excitement!,jumanji,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,tristar pictures teitler film interscope commu...,jumanji adventure fantasy family when siblings...
2,romance comedy,15602.0,grumpier old men,a family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,still yelling. still fighting. still ready for...,grumpier old men,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,warner bros. lancaster gate,grumpier old men romance comedy a family weddi...
3,comedy drama romance,31357.0,waiting to exhale,"cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,friends are the people who let you be yourself...,waiting to exhale,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,twentieth century fox film corporation,waiting to exhale comedy drama romance cheated...
4,comedy,11862.0,father of the bride part ii,just when george banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,just when his world is back to normal... he's ...,father of the bride part ii,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sandollar productions touchstone pictures,father of the bride part ii comedy just when g...


In [53]:
after_users = df_filtered['userId'].nunique()
after_movies = df_filtered['movieId'].nunique()
print(after_movies, after_users)

294 236


In [54]:
movie_thresh = 30
user_thresh = 30

def filter_ratings_iterative(ratings, movie_thresh=30, user_thresh=30):
    """Filtra iterativamente até que todos os filmes tenham pelo menos
    movie_thresh avaliações e todos os usuários tenham pelo menos
    user_thresh avaliações, com base no conjunto atual de ratings."""
    df = ratings.copy()
    while True:
        prev_shape = df.shape[0]
        # filtrar filmes
        movie_counts = df['movieId'].value_counts()
        valid_movies = movie_counts[movie_counts >= movie_thresh].index
        df = df[df['movieId'].isin(valid_movies)]
        # filtrar usuarios
        user_counts = df['userId'].value_counts()
        valid_users = user_counts[user_counts >= user_thresh].index
        df = df[df['userId'].isin(valid_users)]
        # parar se convergiu
        if df.shape[0] == prev_shape:
            break
    return df

df_movies['id'] = df_movies['id'].astype(int)
movies_with_content = set(df_movies['id'].unique())

df_ratings = df_ratings.copy()
df_ratings['movieId'] = df_ratings['movieId'].astype(int)
df_ratings_content = df_ratings[df_ratings['movieId'].isin(movies_with_content)]

df_filtered = filter_ratings_iterative(df_ratings_content, movie_thresh, user_thresh)

print(
    f'Após filtragem iterativa (>= {movie_thresh} ratings por filme, '
    f'>= {user_thresh} ratings por usuário):'
)
print(
    'users =', df_filtered['userId'].nunique(),
    ', movies =', df_filtered['movieId'].nunique(),
    ', ratings =', df_filtered.shape[0]
)


Após filtragem iterativa (>= 30 ratings por filme, >= 30 ratings por usuário):
users = 311 , movies = 356 , ratings = 23097


In [55]:
# Diagnóstico de consistência entre df_movies (com conteúdo) e df_filtered

movies_in_filtered = set(df_filtered['movieId'].astype(int).unique())
movies_in_df_movies = set(df_movies['id'].astype(int).unique())

print('Número de filmes em df_filtered:', len(movies_in_filtered))
print('Número de filmes em df_movies   :', len(movies_in_df_movies))
print('Todos os filmes de df_filtered existem em df_movies?:', movies_in_filtered.issubset(movies_in_df_movies))

only_in_movies = sorted(movies_in_df_movies - movies_in_filtered)
print('Exemplos (até 20) - em df_movies mas não em df_filtered:', only_in_movies[:20])


Número de filmes em df_filtered: 356
Número de filmes em df_movies   : 45433
Todos os filmes de df_filtered existem em df_movies?: True
Exemplos (até 20) - em df_movies mas não em df_filtered: [np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(18), np.int64(20), np.int64(24), np.int64(26), np.int64(27), np.int64(28), np.int64(30), np.int64(33), np.int64(35), np.int64(38), np.int64(55), np.int64(59), np.int64(63), np.int64(64), np.int64(65), np.int64(66)]


In [56]:
# Alinhar df_movies aos filmes que permaneceram em df_filtered
common_ids = sorted(movies_in_filtered)
df_movies = df_movies[df_movies['id'].isin(common_ids)].reset_index(drop=True)

print('df_movies final:', df_movies.shape)
print('df_filtered final:', df_filtered.shape)


df_movies final: (356, 34)
df_filtered final: (23097, 3)


In [57]:
# Verifica igualdade e mostra diferenças (mostra até 20 exemplos)
# Garante tipos inteiros antes de comparar
df_filtered['movieId'] = df_filtered['movieId'].astype(int)
df_movies['id'] = df_movies['id'].astype(int)

movies_in_filtered = set(df_filtered['movieId'].unique())
movies_in_df_movies = set(df_movies['id'].unique())

print('Número de filmes em df_filtered:', len(movies_in_filtered))
print('Número de filmes em df_movies   :', len(movies_in_df_movies))
print('Conjuntos iguais?:', movies_in_filtered == movies_in_df_movies)

only_in_filtered = sorted(movies_in_filtered - movies_in_df_movies)
only_in_df_movies  = sorted(movies_in_df_movies - movies_in_filtered)

print('Exemplos (até 20) - em df_filtered mas não em df_movies:', only_in_filtered[:20])
print('Exemplos (até 20) - em df_movies    mas não em df_filtered:', only_in_df_movies[:20])

Número de filmes em df_filtered: 356
Número de filmes em df_movies   : 356
Conjuntos iguais?: True
Exemplos (até 20) - em df_filtered mas não em df_movies: []
Exemplos (até 20) - em df_movies    mas não em df_filtered: []


In [58]:

df_movies.to_csv('movies_final_df.csv', index=False, encoding='utf-8')
df_filtered.to_csv('ratings_final_df.csv', index=False, encoding='utf-8')

In [ ]:
import os
import time
import requests
from dotenv import load_dotenv

load_dotenv()
final_df = pd.read_csv("movies_final_df.csv")
df_movies = final_df.copy()
TMDB_API_KEY = os.getenv("TMDB_API_KEY")
if not TMDB_API_KEY:
    raise RuntimeError("Defina TMDB_API_KEY no .env como TMDB_API_KEY=...")

TMDB_API_BASE = "https://api.themoviedb.org/3"


def fetch_tmdb_movie(tmdb_id: int) -> dict | None:
    """
    Busca os dados de um filme na API do TMDB.
    Retorna o JSON ou None em caso de 404.
    """
    url = f"{TMDB_API_BASE}/movie/{tmdb_id}"
    params = {
        "api_key": TMDB_API_KEY,
        "language": "en-US",
    }
    resp = requests.get(url, params=params, timeout=10)
    if resp.status_code == 404:
        # filme não encontrado no TMDB
        return None
    resp.raise_for_status()
    return resp.json()


# --- Partimos do df_movies que você já construiu no notebook ---
# Se quiser garantir, pode recarregar:
# df_movies = pd.read_csv("movies_final_df.csv")

new_poster_paths = []
updated = 0
not_found = 0
errors = 0

for idx, row in df_movies.iterrows():
    tmdb_id = int(row["id"])
    old_poster = row.get("poster_path")

    try:
        data = fetch_tmdb_movie(tmdb_id)
        if data is None:
            # filme não existe mais / não encontrado
            new_poster_paths.append(old_poster)
            not_found += 1
            print(f"[NOT FOUND] TMDB id={tmdb_id} (mantendo poster antigo)")
            continue

        new_poster = data.get("poster_path")

        if new_poster:
            new_poster_paths.append(new_poster)
            if new_poster != old_poster:
                updated += 1
                print(f"[OK] id={tmdb_id} atualizou poster: {old_poster} -> {new_poster}")
        else:
            # TMDB não tem poster pra esse filme → mantém o antigo
            new_poster_paths.append(old_poster)
            not_found += 1
            print(f"[NO POSTER] TMDB id={tmdb_id} sem poster (mantendo antigo)")

    except Exception as e:
        errors += 1
        new_poster_paths.append(old_poster)
        print(f"[ERRO] TMDB id={tmdb_id}: {e}")

    # pra ser bonzinho com rate limit
    time.sleep(0.25)

# atualiza a coluna com os novos valores
df_movies["poster_path"] = new_poster_paths

# remove coluna de backup antiga se existir
df_movies.drop(columns=["poster_path", "old_poster"], errors="ignore", inplace=True)

print("\nResumo atualização de posters")
print("-----------------------------")
print("Total de filmes         :", len(df_movies))
print("Posters atualizados     :", updated)
print("Sem poster na API/404   :", not_found)
print("Erros de requisição     :", errors)

print(df_movies[["id", "title", "poster_url"]].head())
df_movies.to_csv("movies_final_df.csv", index=False, encoding="utf-8")


[OK] id=1408 atualizou poster: /odM9973kIv9hcjfHPp6g6BlyTIJ.jpg -> /hYdeBZ4BFXivdouxLfQGWNE6zRx.jpg
[OK] id=5 atualizou poster: /eQs5hh9rxrk1m4xHsIz1w11Ngqb.jpg -> /75aHn1NOYXh4M7L5shoeQ6NGykP.jpg
[OK] id=902 atualizou poster: /eVo6ewq4akfyJYy3GXkMsLNzEJc.jpg -> /whwT3Q9JxbAYzEc3t7uYYcCbTMf.jpg
[OK] id=2054 atualizou poster: /jYEKdpVvRV3KdyHMUVYyg0KTVSY.jpg -> /kBmwWeSN3ajnYHZ3dSybEobxEPI.jpg
[OK] id=628 atualizou poster: /hldXwwViSfHJS0kIJr07KBGmHJI.jpg -> /2162lAT2MP36MyJd2sttmj5du5T.jpg
[OK] id=11 atualizou poster: /btTdmkgIvOi0FFip1sPuZI2oQG6.jpg -> /6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg
[OK] id=527 atualizou poster: /8QF48QS5lvtrhOclt22RFz3bSQm.jpg -> /1nd4SsytVc96hy92g8NNVPD3mzf.jpg
[OK] id=110 atualizou poster: /77CFEssoKesi4zvtADEpIrSKhA3.jpg -> /JHmsBiX1tjCKqAul1lzC20WcAW.jpg
[OK] id=193 atualizou poster: /wjrXjlNpDq9U8vYmAwf420yDFtn.jpg -> /gh0ZZRwSmlzEZTIZee3ZCN9Jssx.jpg
[OK] id=2064 atualizou poster: /9StV7DX4ZtRZGJs9VcVEPvxIKL6.jpg -> /qNGO3ETcNwlWqK2kNRpbJSJRlos.jpg
[OK] id=236

KeyboardInterrupt: 